In [33]:
import torch
import pandas as pd
from utils import Logger
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from data_treatment import CreditCardDataSet
from discriminator import *
from generator import *
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import glob

In [34]:
# Noise
def noise(size):
    n = Variable(torch.randn(size, 100))
    if torch.cuda.is_available(): 
        return n.cuda() 
    return n

In [45]:
files_dropdown = widgets.Dropdown(
    options=glob.glob("*.csv"),
    description='File:',
    value = 'diabetes.csv',
    disabled=False,
)
display(files_dropdown)

Dropdown(description='File:', index=2, options=('creditcard.csv', 'data.csv', 'diabetes.csv'), value='diabetes…

In [46]:
file_name=files_dropdown.value
if file_name == "data.csv":
    message = "Breast Cancer Wisconsin (Diagnostic) Data Set"
    class_name = "diagnosis"
    values_names = {0: "Benign", 1: "Malignant"}
    class_len = 33
elif file_name == "creditcard.csv":
    message = "Credit Card Fraud Detection"
    class_name = "Class"
    values_names = {0: "No Frauds", 1: "Frauds"}
    class_len = 31
elif file_name == "diabetes.csv":
    message="Pima Indians Diabetes Database"
    class_name = "Outcome"
    values_names = {0: "Normal", 1: "Diabets"}
    class_len = 9
else:
     exit()

In [47]:
def train_generator(optimizer, fake_data):
    # 2. Train Generator
    # Reset gradients
    optimizer.zero_grad()
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    # Calculate error and backpropagate
    error = loss(prediction, real_data_target(prediction.size(0)))
    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

def train_discriminator(optimizer, real_data, fake_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = discriminator(real_data)
    # Calculate error and backpropagate
    error_real = loss(prediction_real, real_data_target(real_data.size(0)))
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, fake_data_target(real_data.size(0)))
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error
    return error_real + error_fake, prediction_real, prediction_fake

In [48]:
def real_data_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

def fake_data_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

In [49]:
lr = 0.002
#lr= 0.0002
d_steps = 1
batch_size = 5
num_epochs = 1000
print_interval = 100
num_test_samples = 16
loss = nn.BCELoss()

In [50]:
discriminator = DiscriminatorNet(class_len)
generator = GeneratorNet(class_len)
if torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()
d_optimizer = optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = optim.Adam(generator.parameters(), lr=lr)

In [51]:

test_noise = noise(num_test_samples)
logger = Logger(model_name=file_name[:-4], data_name=file_name[:-4] + "_" + class_name)
database = CreditCardDataSet (csv_file=file_name, root_dir=".")

data_loader = torch.utils.data.DataLoader(database, batch_size=batch_size, shuffle=True)

num_batches = len(data_loader)

In [52]:
for epoch in range(num_epochs):
    print("Epoch ", epoch)

    for n_batch, real_batch in enumerate(data_loader):
        # 1. Train Discriminator
        real_data = Variable(real_batch).float()
        if torch.cuda.is_available(): 
            real_data = real_data.cuda()
        # Generate fake data
        fake_data = generator(noise(real_data.size(0))).detach()
        # Train D
        d_error, d_pred_real, d_pred_fake = train_discriminator(d_optimizer,
                                                                real_data, fake_data)

        # 2. Train Generator
        # Generate fake data
        fake_data = generator(noise(real_batch.size(0)))
        # Train G
        g_error = train_generator(g_optimizer, fake_data)
        # Log error
        logger.log(d_error, g_error, epoch, n_batch, num_batches)

        # Display Progress

        if (n_batch) % print_intervaç == 0:
            filename = "results/" + file_name[:-4] + "/" + str(epoch) + "_" + str(n_batch) + ".txt"
            print(filename)
            file = open(filename, "w")
            print("Discriminator error: ", d_error)
            print("Generator error: ", g_error)
            print("")

            file.write("Discriminator error: " + str(d_error) + "\n")
            file.write("Generator error: " + str(g_error) + "\n")
            file.write("Points: " + str(fake_data) + "\n\n\n")
        #     #display.clear_output(True)
        #     # Display Images
        #     test_images = generator(test_noise).data.cpu()
        #     logger.log_images(test_images, num_test_samples, epoch, n_batch, num_batches);
        #     # Display status Logs
        #     #logger.display_status(epoch, num_epochs, n_batch, num_batches, d_error, g_error, d_pred_real, d_pred_fake)
        # # Model Checkpoints

        logger.save_models(generator, discriminator, epoch)

Epoch  0
results/diabetes/0_0.txt
Discriminator error:  tensor(0.8150, device='cuda:0', grad_fn=<ThAddBackward>)
Generator error:  tensor(0.7211, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

results/diabetes/0_50.txt
Discriminator error:  tensor(27.6310, device='cuda:0', grad_fn=<ThAddBackward>)
Generator error:  tensor(0., device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

results/diabetes/0_100.txt
Discriminator error:  tensor(27.6310, device='cuda:0', grad_fn=<ThAddBackward>)
Generator error:  tensor(0., device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

results/diabetes/0_150.txt
Discriminator error:  tensor(27.6310, device='cuda:0', grad_fn=<ThAddBackward>)
Generator error:  tensor(0., device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

Epoch  1
results/diabetes/1_0.txt
Discriminator error:  tensor(27.6310, device='cuda:0', grad_fn=<ThAddBackward>)
Generator error:  tensor(0., device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)



KeyboardInterrupt: 